In [31]:
import pandas as pd
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.models import Sequential
from keras.layers import Dense, Dropout
from sklearn.model_selection import train_test_split
from keras.utils.np_utils import to_categorical
from google.colab import drive
import keras

In [2]:
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
!ls "/content/drive/My Drive/TwitterSentimentDL/Data"

test.csv  train.csv


# Load Data with pandas

In [53]:
train_data = pd.read_csv("/content/drive/My Drive/TwitterSentimentDL/Data/train.csv")
test_data = pd.read_csv("/content/drive/My Drive/TwitterSentimentDL/Data/test.csv")

# Format Data

In [54]:
train_data = train_data[['text', 'sentiment']]
test_data = test_data[['text','sentiment']]

In [55]:
print(train_data['text'])

0                      I`d have responded, if I were going
1            Sooo SAD I will miss you here in San Diego!!!
2                                my boss is bullying me...
3                           what interview! leave me alone
4         Sons of ****, why couldn`t they put them on t...
                               ...                        
27476     wish we could come see u on Denver  husband l...
27477     I`ve wondered about rake to.  The client has ...
27478     Yay good for both of you. Enjoy the break - y...
27479                           But it was worth it  ****.
27480       All this flirting going on - The ATG smiles...
Name: text, Length: 27481, dtype: object


In [56]:
import re

train_data['text'] = train_data['text'].apply(lambda x: str(x).lower())
X_train = train_data['text'].apply((lambda x: re.sub('[^a-zA-z0-9\s]','',str(x))))

test_data['text'] = test_data['text'].apply(lambda x: str(x).lower())
X_test = test_data['text'].apply((lambda x: re.sub('[^a-zA-z0-9\s]','',str(x))))


In [57]:
print(X_train[1])

 sooo sad i will miss you here in san diego


In [58]:
from sklearn.feature_extraction.text import TfidfVectorizer

In [59]:
tokenization = TfidfVectorizer(max_features=3500)

tokenization.fit(X_train)
X_train = tokenization.transform(X_train).toarray()

tokenization.fit(X_test)
X_test = tokenization.transform(X_test).toarray()

In [69]:
import numpy as np

sentiment = {'negative': 0,'neutral': 1, 'positive' : 2} 

Y_train = np.array([sentiment[x] for x in train_data['sentiment']])
Y_test = np.array([sentiment[x] for x in test_data['sentiment']])
print(Y_train)

[1 0 0 ... 2 2 1]


# Constructing Model

In [61]:
model = Sequential()

In [63]:
model.add(Dense(512, input_shape=(X_train.shape[1],), activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(256, activation='sigmoid'))
model.add(Dropout(0.5))
model.add(Dense(3, activation='softmax'))

In [64]:
print(model.summary())

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_6 (Dense)              (None, 512)               1792512   
_________________________________________________________________
dropout_4 (Dropout)          (None, 512)               0         
_________________________________________________________________
dense_7 (Dense)              (None, 256)               131328    
_________________________________________________________________
dropout_5 (Dropout)          (None, 256)               0         
_________________________________________________________________
dense_8 (Dense)              (None, 2)                 514       
_________________________________________________________________
dense_9 (Dense)              (None, 512)               1536      
_________________________________________________________________
dropout_6 (Dropout)          (None, 512)              

### Training

In [66]:
batch = 32
print(X_train.shape)
print(Y_train.shape)
optimizer = keras.optimizers.Adam(lr=0.002, beta_1=0.9, beta_2=0.999, epsilon=None, 
                                      decay=0.0, amsgrad=False)

model.compile(loss='sparse_categorical_crossentropy',
                 optimizer=optimizer,
                 metrics=['accuracy'])


model.fit(X_train, Y_train, epochs = 5, batch_size = 32)

(27481, 3500)
(27481,)
Epoch 1/5
859/859 [==============================] - 15s 17ms/step - loss: 0.9156 - accuracy: 0.5577
Epoch 2/5
859/859 [==============================] - 15s 17ms/step - loss: 0.7229 - accuracy: 0.6975
Epoch 3/5
859/859 [==============================] - 15s 17ms/step - loss: 0.6363 - accuracy: 0.7544
Epoch 4/5
859/859 [==============================] - 15s 17ms/step - loss: 0.5516 - accuracy: 0.7980
Epoch 5/5
859/859 [==============================] - 15s 17ms/step - loss: 0.4676 - accuracy: 0.8369


#Testing

In [67]:
test_mse_score, test_mae_score = model.evaluate(X_test, Y_test)

111/111 [==============================] - 1s 6ms/step - loss: 1.9117 - accuracy: 0.3633


In [68]:
print("model  MSE: ")
print(test_mse_score)
print("model MAE: ")
print(test_mae_score)

model  MSE: 
1.9116714000701904
model MAE: 
0.36332768201828003
